In [259]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from skimpy import clean_columns, skim
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.decomposition import TruncatedSVD
from xgboost import XGBRFRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import LinearSVC

In [260]:

brent = clean_columns (pd.read_csv("new-shell-cashflow-datathon-2023/brent.csv", parse_dates= ['Tarih']))
cash_flow_train = clean_columns(pd.read_csv("new-shell-cashflow-datathon-2023/cash_flow_train.csv", parse_dates= ['Date']))  
depo_pump_imm =  clean_columns(pd.read_csv("new-shell-cashflow-datathon-2023/depo_pump_imm.csv")) 
otv =  clean_columns(pd.read_csv("new-shell-cashflow-datathon-2023/otv.csv", parse_dates= ['Tarih'])) 
platts = clean_columns(pd.read_csv("new-shell-cashflow-datathon-2023/platts.csv", parse_dates= ['Tarih']))  
sample_submission = pd.read_csv("new-shell-cashflow-datathon-2023/sample_submission.csv")
usd =  clean_columns(pd.read_csv("new-shell-cashflow-datathon-2023/usd.csv", parse_dates= ['Tarih'])) 
volume = clean_columns(pd.read_csv("new-shell-cashflow-datathon-2023/volume.csv", parse_dates= ['Posting date']))

7 column names have been cleaned

17 column names have been cleaned

5 column names have been cleaned

5 column names have been cleaned

7 column names have been cleaned

8 column names have been cleaned

5 column names have been cleaned

In [261]:
brent_cash = cash_flow_train.merge(brent, left_on= 'date', right_on='tarih', how= 'left').drop('tarih', axis =1)

In [262]:
dates = pd.DataFrame({'date' :pd.date_range(start='1-1-2018', end= '31-12-2022')})  
dates['year'] = dates['date'].dt.year
dates['month'] = dates['date'].dt.month

/var/folders/t1/db5m68vn10v3148_nq_7r8b80000gn/T/ipykernel_998/870535568.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  dates = pd.DataFrame({'date' :pd.date_range(start='1-1-2018', end= '31-12-2022')})


In [263]:
depo = dates.merge(depo_pump_imm, left_on= ['year', 'month'], right_on= ['yl', 'ay']).drop(['yl','ay'], axis = 1)

In [264]:
brent_cash_depo = brent_cash.merge(depo, on = 'date', how = 'left')

In [265]:
otv.drop_duplicates(inplace= True)

In [266]:
otv_fixed =  otv.drop(['otv','adet'], axis = 1).pivot(index= 'tarih', columns= 'otv_uygulanan_urun_ad').reset_index()

In [267]:
otv_fixed = otv_fixed.droplevel(0, axis =1)

In [268]:
otv_fixed.columns = ['date', 'benzinli_motorin', 'etanallu_kursunsuz_benzin_95', 'kursunsuz_benzin_95','lpg','motorin']

In [269]:
otv_fixed.fillna(method= 'bfill').fillna(method= 'ffill', inplace= True )

In [270]:
brent_cash_depo_otv =  brent_cash_depo.merge(otv_fixed,left_on = 'date', right_on = 'date', how = 'left')

In [271]:
platts_1 =  platts[platts['urun'] == '10 ppm ULSD CIF Med (Genova/Lavera)']

In [272]:
platts_2 = platts[platts['urun'] == 'Prem Unl 10 ppm CIF Med (Genova/Lavera)']

In [273]:
brent_cash_depo_otv_platts_1 =  brent_cash_depo_otv.merge(platts_1,left_on = 'date', right_on = 'tarih', how = 'left')

In [274]:
brent_cash_depo_otv_platts =  brent_cash_depo_otv_platts_1.merge(platts_2,left_on = 'date', right_on = 'tarih', how = 'left')

In [275]:
brent_cash_depo_otv_platts_usd =  brent_cash_depo_otv_platts.merge(usd,left_on = 'date', right_on = 'tarih', how = 'left')

In [276]:
brent_cash_depo_otv_platts_usd.drop(['tarih_x', 'tarih_y'], axis= 1, inplace= True)

In [277]:
volume_fixed =  volume.pivot( columns= ['product', 'material_description', 'category'], values=  'sales_volumes_in_l_15', index= 'posting_date')

In [278]:
volume_fixed = volume_fixed.droplevel(level=0, axis='columns')

In [279]:
volume_fixed = volume_fixed.droplevel(level=0, axis='columns')

In [280]:
brent_cash_depo_otv_platts_usd_volume=  brent_cash_depo_otv_platts_usd.merge(volume_fixed, left_on= 'date', right_on= 'posting_date', how= 'left')

In [281]:
brent_cash_depo_otv_platts_usd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1025 entries, 0 to 1024
Data columns (total 53 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            1025 non-null   datetime64[ns]
 1   total_inflows                   1025 non-null   float64       
 2   customers_dds                   1025 non-null   float64       
 3   customers_eft                   1025 non-null   float64       
 4   t_&_s_collections               1025 non-null   float64       
 5   fx_sales                        1025 non-null   float64       
 6   other_operations                1025 non-null   float64       
 7   total_outflows                  1025 non-null   float64       
 8   tupras                          1025 non-null   float64       
 9   other_oil                       1025 non-null   float64       
 10  gas                             1025 non-null   float64       
 11  impo

In [282]:
limit_par = len(brent_cash_depo_otv_platts_usd_volume) * 0.7
brent_cash_depo_otv_platts_usd_volume.dropna(thresh= limit_par, axis=1, inplace= True)

In [283]:
brent_cash_depo_otv_platts_usd_volume['date_'] = brent_cash_depo_otv_platts_usd_volume['date'] + pd.Timedelta(days=105)

In [284]:
brent_cash_depo_otv_platts_usd_volume.set_index('date_', inplace= True)

In [285]:
data = brent_cash_depo_otv_platts_usd_volume._get_numeric_data()

In [286]:
# data['day_of_the_week'] = data.index.day_of_week
# data['month'] = data.index.month

In [287]:
data_train =  data['2019-04-02':'2023-02-01']

In [288]:
data_test = data['2023-02-02': '2023-05-12']

In [289]:
data_test.drop('net_cashflow_from_operations', axis=1, inplace= True)

/var/folders/t1/db5m68vn10v3148_nq_7r8b80000gn/T/ipykernel_998/2263494553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test.drop('net_cashflow_from_operations', axis=1, inplace= True)


In [290]:
X = data_train.drop('net_cashflow_from_operations', axis=1)

In [291]:
y = data_train['net_cashflow_from_operations']

In [292]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42, shuffle=False)

In [293]:
pipe = Pipeline([('impute', KNNImputer()), 
                 ('onehot', OneHotEncoder(handle_unknown='ignore')),
                  ('rf', RandomForestRegressor(random_state = 42, 
                                               n_estimators= 1000,
                                               min_samples_leaf = 10))])

In [294]:
np.random.seed(42)
pipe.fit(X,y)

Pipeline(steps=[('impute', KNNImputer()),
                ('onehot', OneHotEncoder(handle_unknown='ignore')),
                ('rf',
                 RandomForestRegressor(min_samples_leaf=10, n_estimators=1000,
                                       random_state=42))])

In [295]:
np.random.seed(42)
prediction = pipe.predict(X_test)

In [296]:
mean_absolute_error(y_test, prediction)

55103252.58941893

In [297]:
np.random.seed(42)
sub = pipe.predict(data_test)

In [298]:
submission = pd.DataFrame({'Date' :  data_test.index, 'Net Cashflow from Operations' : sub})

In [299]:
submission['Date'] = submission['Date'].astype('str')
submission['Net Cashflow from Operations'] = submission['Net Cashflow from Operations'].astype('int64')

In [300]:
submission.drop(labels= [56,62], axis= 0, inplace= True)

In [301]:
pd.DataFrame.to_csv(submission, 'submission.csv', index= False)

In [302]:
submission

,Date,Net Cashflow from Operations
0,2023-02-02,55126627
1,2023-02-03,-7217432
2,2023-02-06,693454
3,2023-02-07,29147437
4,2023-02-08,55749260
...,...,...
67,2023-05-08,2612296
68,2023-05-09,4350412
69,2023-05-10,-1884595
70,2023-05-11,-515113
